## Google Dance

In [1]:
#Importing all the libraries required for scraping

from splinter import Browser
from bs4 import BeautifulSoup 
import requests
import time
import pandas as pd
import pprint
import webbrowser
import urllib
import re
import pymongo
from geopy.geocoders import Nominatim  

In [2]:
# using the crome driver for scraping

executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
#browser = Browser()

In [3]:
# Url for main page
url = 'https://www.google.com/search?rlz=1C1CHBF_enUS853US853&sz=0&tbm=lcl&ei=Y5HcXZW5CdG05gKT7KzIAw&q=kids+dance+classes+new+jersey&oq=kids+dance+classes+new+jersey&gs_l=psy-ab.3...140911.143275.0.143677.6.6.0.0.0.0.271.904.0j4j1.5.0....0...1c.1.64.psy-ab..1.0.0....0.2IUp-hMGy8U#rlfi=hd:;si:;mv:[[41.0322952,-73.93508229999999],[40.534548400000006,-74.637359]];tbs:lrf:!1m4!1u3!2m2!3m1!1e1!1m4!1u2!2m2!2m1!1e1!2m1!1e2!2m1!1e3!3sIAE,lf:1,lf_ui:2'
browser.visit(url)

# updating the page by clicking on search button
browser.find_by_name('btnG').first.click()

# parser
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
time.sleep(5)

# storing all the data from the main class
Data = soup.find_all("div",attrs = {"class": "VkpGBb"})

print(len(Data))
top_result = []
    
# search variables Name and reviews within main class using for loop
for tr in Data:
    dataframe = {}
    dataframe["name"] = (tr.find("div",attrs = {"class": "dbg0pd"})).text.replace('\n', ' ')
    #dataframe["Reviews"] = (tr.find('span', {'class':'BTtC6e'}))
    top_result.append(dataframe)
print(top_result)

22
[{'name': "Sharon's Studio of Dance & Music"}, {'name': 'Grooves Unlimited Dance Studio'}, {'name': 'Wild Child Dance Studio'}, {'name': 'Hudson Dance Studio Edgewater'}, {'name': 'Class Act Performing Arts Studio'}, {'name': "Sharon's Studio of Dance & Music"}, {'name': 'Grooves Unlimited Dance Studio'}, {'name': 'Harmony Dance Center'}, {'name': 'Dance Palooza Studios'}, {'name': 'Dance Academy of North Jersey'}, {'name': 'Starlight Dance Center'}, {'name': 'Village Dance Academy'}, {'name': 'Kaye Lynn Dance Studio I'}, {'name': 'DDA Dance Academy'}, {'name': 'New Jersey Ballet Company and School'}, {'name': 'DanceWorks Studios'}, {'name': 'Uptown Dance Studio'}, {'name': 'The Kennedy Dancers'}, {'name': 'bboy bgirl lifestyle'}, {'name': 'Dance Theatre of New Jersey'}, {'name': 'Next Step Broadway'}, {'name': 'Performing Arts Workshop'}]


In [4]:

Data = []
# Using the geo locator for getting Lat and Long
geolocator = Nominatim()

# getting the link for next pages 

pageData = soup.find_all(class_='std')

print(len(pageData))

for tr in pageData:
    
    pLink = str(tr.find('a')['href'])
    print(pLink)

#updating each page link with specific nunmbers to go to next page

top_result = []
list1 = [2,4,6,8,10,12] 
for x in (list1):
    prePage = x
    #print(prePage)
    startPage = "start="+ str(prePage) +"0"
    #print(startPage)
    replacePage = "start="+ str(prePage+2) +"0"
    #print(replacePage)
    pLink = pLink.replace(startPage,replacePage)
    pageLink = "https://www.google.com" + pLink
    #print(pageLink)
    browser.visit(pageLink)

# parsing pages as the pages updated with above url

    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    Data = soup.find_all("div",attrs = {"class": "VkpGBb"})

    print(len(Data))

# search variables Name, url, adress, phone no, zipcode within main class using for loop

    for tr in Data:
        
        dataframe = {}
        dataframe["name"] = (tr.find("div",attrs = {"class": "dbg0pd"})).text.replace('\n', ' ')
        #dataframe["Reviews"] = (tr.find('span', {'class':'BTtC6e'}))
        dataframe["type"] = "Dance Classes"
        
#if the url is empty handling the exception

        try:
            dataframe["url"] = (tr.find("a",attrs = {"class": "yYlJEf L48Cpd"})['href'])
            
            print(dataframe["url"])
        except:
            print('Cant Find the url')

# scraping for phone number

        PhoneData = tr.find('span').text.split(' ')
        #print(PhoneData)
        lPhoneData = len(PhoneData)
        phoneNo = PhoneData[lPhoneData-2]+ PhoneData[lPhoneData-1]
        phoneNo = re.sub("\D", "", phoneNo)
        dataframe["phoneNo"] = phoneNo
        
        dirClass = (tr.find('div', {'class':'UbRuwe'}))
        #print(dirClass)
        
# if the main directory is empty, handling the exception
        if dirClass != None :
        #if hasattr('a', 'cursor:pointer'):
        
# getting the address
            rawAddress = (tr.find("a",attrs = {"class": "yYlJEf VByer"})["data-url"])

# cleaning the address based on number of delimters

            orgAddress = rawAddress.split('/')[4].strip()
            #print(orgAddress)
            orgAddress = orgAddress.replace('+',' ')
            orgAddress = orgAddress.split(',')
            
            #print(orgAddress)
            lenAddress = len(orgAddress)
            
            if lenAddress == 4 :
                ZipCode = orgAddress[3].split(' ')
                #print(ZipCode)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[1]+', ' + orgAddress[2]+', ' + ZipCode[1]+', ' + ZipCode[2]
                #dataframe["city"] = orgAddress[2]
                #dataframe["state"] = ZipCode[1]
                #if len(ZipCode) > 2:
                #    dataframe["zipcode"] = ZipCode[2]
                #    dataframe["Zipcode"] = orgAddress[4]
                    
            if lenAddress > 4 :
                ZipCode = orgAddress[lenAddress-1].split(' ')
                #print(ZipCode)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[lenAddress-3]+', '+orgAddress[lenAddress-2]+', '+ ZipCode[1]+', '+ZipCode[2]
                #dataframe["city"] = orgAddress[lenAddress-2]
                #dataframe["state"] = ZipCode[1]
                #if len(ZipCode) > 2:
                #    dataframe["zipcode"] = ZipCode[2]
                #    dataframe["Zipcode"] = orgAddress[4]
            
                
            if lenAddress < 4 :
                ZipCode = orgAddress[lenAddress-1].split(' ')
                #print(orgAddress)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[lenAddress-2]+', '+orgAddress[lenAddress-1]+', '+ ZipCode[1]+', '+ZipCode[2]
                #dataframe["city"] = orgAddress[lenAddress-1]
                #dataframe["state"] = ZipCode[1]
                #if len(ZipCode) > 2:
                #    dataframe["zipcode"] = ZipCode[2]
                #    dataframe["Zipcode"] = orgAddress[4]
            
                #print(orgAddress)
                
# Using geolocator to find the lat and long for each address

            try:
                locForLatLong = dataframe["address"] #+ dataframe["city"]
                #print(locForLatLong)
                location = geolocator.geocode(locForLatLong, timeout=10)
                #print(location.address)
                #print((location.latitude,location.longitude))
                dataframe["location"] = [location.latitude,location.longitude]
                
            except AttributeError:
                print('Cant Find the address')
                
                
        top_result.append(dataframe)

#print(top_result)

# creating a dataframe for Dance
df1=pd.DataFrame(top_result)



1
/search?q=kids+dance+classes+new+jersey&rlz=1C1CHBF_enUS853US853&sz=0&tbm=lcl&ei=OaTuXdK4L8KIggeFjYqgDw&start=20&sa=N&ved=0ahUKEwiSuYqLqqnmAhVChOAKHYWGAvQQ8tMDCOQC


C:\Users\13157\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


20
http://www.newarkschoolofthearts.org/
http://dynamicdancenj.com/
http://dancestarznj.net/
http://www.movementspace.com/
http://showbizdancestudio.com/classes/
Cant Find the address
http://www.nrityacreations.com/
http://www.centerstagedancenj.com/
http://www.bergencountydanceclasses.com/
http://www.progressivedancestudio.com/
Cant Find the url
http://www.gardenstreetmusic.com/
Cant Find the address
http://harmonydancetheater.com/
http://www.dwacademy.com/
http://www.danceworksdenville.com/
Cant Find the url
http://www.dwacademy.com/
http://www.smapa.org/
http://www.jerseycitydanceacademy.com/
http://www.broadwaydancetheatre.com/
http://www.ucdconline.com/
20
http://artistsinmotionnj.com/
Cant Find the address
http://www.thewestfieldschoolofdance.com/
http://www.cjdance-gym.com/
Cant Find the address
http://www.aryaintl.com/
http://www.dancetcstudio.com/
http://www.balletartsnj.com/
http://www.gotta-dance.com/
http://www.tatedanceacademy.com/
Cant Find the url
http://www.kidville.com

In [5]:
#df1.to_excel("outputDance.xlsx")

# creating a dataframe for Dance
df1


,name,type,url,phoneNo,address,location
0,Newark School of the Arts,Dance Classes,http://www.newarkschoolofthearts.org/,,"89 Lincoln Park, Newark, NJ, 07102","[40.7256942631579, -74.1794780526316]"
1,Dynamic Dance Academy,Dance Classes,http://dynamicdancenj.com/,,"546 Bloomfield Ave, Verona, NJ, 07044","[40.8345572, -74.2564094]"
2,Dance Starz Academy of NJ,Dance Classes,http://dancestarznj.net/,,"496 Inman Ave, Rahway, NJ, 07065","[40.6001789, -74.3113377]"
3,Monroe Street Movement Space,Dance Classes,http://www.movementspace.com/,,"720 Monroe St, Hoboken, NJ, 07030","[40.7466675918367, -74.0378767142857]"
4,Showbiz Kidz Dance Center,Dance Classes,http://showbizdancestudio.com/classes/,,"408 NJ-57, Phillipsburg, NJ, 08865",NaN
...,...,...,...,...,...,...
115,Simply Dance,Dance Classes,http://www.simplydanceonline.com/,330,"3 Church St, Vernon Township, NJ, 07462",NaN
116,"Center Stage Dance, Theatre, Music & Gymnastic...",Dance Classes,http://www.centerstagenj.com/,9,"10 Alvin Ct %23104, East Brunswick, NJ, 08816",NaN
117,"Art of Motion, Inc.",Dance Classes,http://www.artofmotion.org/,8,NaN,NaN
118,Broadway Bound Dance Center,Dance Classes,http://bbdcnj.com/,9,"1245 US-22, Lebanon, NJ, 08833","[40.6436680541245, -74.8308720222854]"


## Google Music

In [6]:
url = 'https://www.google.com/search?rlz=1C1CHBF_enUS853US853&tbm=lcl&sxsrf=ACYBGNTdFaDd4jAeC6lZKeILHtxz3pMx4A%3A1574389394556&ei=kkbXXanHIeSV_Qbz-a_wAg&q=kids+music+classes+new+jersey&oq=kids+music+classes+new+jersey&gs_l=psy-ab.12...0.0.0.163915.0.0.0.0.0.0.0.0..0.0....0...1c..64.psy-ab..0.0.0....0.51R-_vCUfl4#rlfi=hd:;si:;mv:[[41.0328425,-73.9399237],[40.1005597,-74.3127616]];tbs:lrf:!1m4!1u3!2m2!3m1!1e1!1m4!1u2!2m2!2m1!1e1!1m4!1u16!2m2!16m1!1e1!1m4!1u16!2m2!16m1!1e2!2m1!1e2!2m1!1e3!2m1!1e16!3sIAE,lf:1,lf_ui:2'
browser.visit(url)

browser.find_by_name('btnG').first.click()

html = browser.html
soup = BeautifulSoup(html, 'html.parser')
time.sleep(5)

Data = soup.find_all("div",attrs = {"class": "VkpGBb"})

print(len(Data))
top_result = []
    
for tr in Data:
    dataframe = {}
    dataframe["name"] = (tr.find("div",attrs = {"class": "dbg0pd"})).text.replace('\n', ' ')
    #dataframe["Reviews"] = (tr.find('span', {'class':'BTtC6e'}))
    top_result.append(dataframe)
print(top_result)

22
[{'name': 'West Essex Music Together'}, {'name': 'Rockness Music of Montclair'}, {'name': 'The Institute of Music for Children'}, {'name': 'Musical Munchkins of New Jersey'}, {'name': 'Little Rockers - cool kids music classes & parties'}, {'name': 'Music Together of Bergen County'}, {'name': 'Rockness Music of Jersey City'}, {'name': 'Little Rockers - cool kids music classes & parties'}, {'name': 'Rockness Music of Montclair'}, {'name': 'Lindeblad School of Music'}, {'name': 'Joy Kids Music & Art (Joy Performing Art Center)'}, {'name': 'Little Notes Music School'}, {'name': "Sharon's Studio of Dance & Music"}, {'name': 'Music Together of Montclair'}, {'name': 'The Extension Division at Cali School of Music'}, {'name': "Kids' MusicRound"}, {'name': 'West Essex Music Together'}, {'name': 'Do Re Mi School of Music and the Arts'}, {'name': 'Edison School of Music and Dance'}, {'name': 'New Jersey Conservatory'}, {'name': 'A Sound Start'}, {'name': 'Music Together of Summit'}]


In [7]:
Data = []
geolocator = Nominatim()
pageData = soup.find_all(class_='std')

print(len(pageData))
for tr in pageData:
    
    pLink = str(tr.find('a')['href'])
    print(pLink)

top_result = []
list1 = [2,4,6,8,10,12] 
for x in (list1):
    prePage = x
    #print(prePage)
    startPage = "start="+ str(prePage) +"0"
    #print(startPage)
    replacePage = "start="+ str(prePage+2) +"0"
    #print(replacePage)
    pLink = pLink.replace(startPage,replacePage)
    pageLink = "https://www.google.com" + pLink
    #print(pageLink)
    browser.visit(pageLink)

  
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    Data = soup.find_all("div",attrs = {"class": "VkpGBb"})

    print(len(Data))
    
    for tr in Data:
        
        dataframe = {}
        dataframe["name"] = (tr.find("div",attrs = {"class": "dbg0pd"})).text.replace('\n', ' ')
        #dataframe["Reviews"] = (tr.find('span', {'class':'BTtC6e'}))
        dataframe["type"] = "Music Classes"
        try:
            dataframe["url"] = (tr.find("a",attrs = {"class": "yYlJEf L48Cpd"})['href'])
            
            print(dataframe["url"])
        except:
            print('Cant Find the url')
            
        
        PhoneData = tr.find('span').text.split(' ')
        #print(PhoneData)
        lPhoneData = len(PhoneData)
        phoneNo = PhoneData[lPhoneData-2]+ PhoneData[lPhoneData-1]
        phoneNo = re.sub("\D", "", phoneNo)
        dataframe["phoneNo"] = phoneNo
        
        dirClass = (tr.find('div', {'class':'UbRuwe'}))
        #print(dirClass)
        if dirClass != None :
        #if hasattr('a', 'cursor:pointer'):
            rawAddress = (tr.find("a",attrs = {"class": "yYlJEf VByer"})["data-url"])
            
            orgAddress = rawAddress.split('/')[4].strip()
            #print(orgAddress)
            orgAddress = orgAddress.replace('+',' ')
            orgAddress = orgAddress.split(',')
            
            #print(orgAddress)
            lenAddress = len(orgAddress)
            
            if lenAddress == 4:
                ZipCode = orgAddress[3].split(' ')
                #print(ZipCode)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[1]+', ' + orgAddress[2]+', ' + ZipCode[1]+', ' + ZipCode[2]
                #dataframe["city"] = orgAddress[2]
                #dataframe["state"] = ZipCode[1]
                #if len(ZipCode) > 2:
                    #dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
                    
            if lenAddress > 4 :
                ZipCode = orgAddress[lenAddress-1].split(' ')
                #print(ZipCode)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[lenAddress-3]+', '+orgAddress[lenAddress-2]+', '+ ZipCode[1]+', '+ZipCode[2]
                #dataframe["city"] = orgAddress[lenAddress-2]
                #dataframe["state"] = ZipCode[1]
                #if len(ZipCode) > 2:
                    #dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
            
                
            if lenAddress < 4 :
                ZipCode = orgAddress[lenAddress-1].split(' ')
                #print(orgAddress)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[lenAddress-2]+', '+orgAddress[lenAddress-1]+', '+ ZipCode[1]+', '+ZipCode[2]
                #dataframe["city"] = orgAddress[lenAddress-1]
                #dataframe["state"] = ZipCode[1]
                #if len(ZipCode) > 2:
                    #dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
            
                
                #print(orgAddress)
            try:
                locForLatLong = dataframe["address"] #+ dataframe["city"]
                #print(locForLatLong)
                location = geolocator.geocode(locForLatLong, timeout=10)
                #print(location.address)
                #print((location.latitude,location.longitude))
                dataframe["location"] = [location.latitude,location.longitude]
                
            except AttributeError:
                print('Cant Find the address')
                
                
        top_result.append(dataframe)

#print(top_result)
df2=pd.DataFrame(top_result)



1
/search?q=kids+music+classes+new+jersey&rlz=1C1CHBF_enUS853US853&sz=0&tbm=lcl&ei=gKTuXd3wK8i0ggfyjbSYCg&start=20&sa=N&ved=0ahUKEwjdsPSsqqnmAhVImuAKHfIGDaMQ8tMDCK8C


C:\Users\13157\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  


20
http://www.montclairmusic.com/
Cant Find the address
http://jazzberries.yourvirtuoso.com/
http://www.newarkschoolofthearts.org/
http://www.funfamilymusic.com/location.aspx?loc=1718
https://www.rocknessmusic.com/
Cant Find the url
http://www.creativekidsnj.com/
https://wayne.schoolofrock.com/
Cant Find the address
http://www.artkidsacademy.com/
https://kidsmusicround.com/
https://denville.b2rmusic.com/
Cant Find the address
http://www.pyrockmusicschool.com/
Cant Find the address
http://musictogetherfamily.com/
http://www.musicandmorewithkelly.com/
http://saddlebrook.schoolofrock.com/
http://www.musictogetherofnorthernnj.com/
http://www.jazzhousekids.org/
http://www.octopusmusicschool.com/
http://www.kidsmusicround.com/
http://www.pencilworksartstudio.com/
20
http://www.firstnotes.com/
http://www.newjerseyguitarlessons.com/
https://clark.schoolofrock.com/
http://www.whartonarts.org/
https://chatham.schoolofrock.com/
https://www.calderoneschoolofmusic.com/
http://jerseycitymusiclessons

In [8]:
#df.to_excel("outputMusic.xlsx")

# Creating a dataframe for Music
df2

,name,type,url,phoneNo,address,location
0,Montclair Music Studio,Music Classes,http://www.montclairmusic.com/,8,"104 Watchung Ave, Upper Montclair, NJ, 07043",NaN
1,Kindermusik with Susan Leone,Music Classes,http://jazzberries.yourvirtuoso.com/,9737626575,"516 Prospect St, Maplewood, NJ, 07040","[40.734023, -74.263358]"
2,Newark School of the Arts,Music Classes,http://www.newarkschoolofthearts.org/,,"89 Lincoln Park, Newark, NJ, 07102","[40.7256942631579, -74.1794780526316]"
3,Music Together of Warren Township,Music Classes,http://www.funfamilymusic.com/location.aspx?lo...,9086560129,"39 Maple Ave, Morristown, NJ, 07960","[40.793756, -74.480783]"
4,Rockness Music of Scotch Plains,Music Classes,https://www.rocknessmusic.com/,,"18 Cliffwood St, Scotch Plains, NJ, 07076","[40.635383, -74.37265]"
...,...,...,...,...,...,...
115,New Jersey School of Music - Medford,Music Classes,http://www.njschoolofmusic.com/,9,"10 Union St, Medford, NJ, 08055","[39.899117264414, -74.8263799430322]"
116,Real Brave,Music Classes,https://realbraveaudio.com/,9,"350 Ramapo Valley Rd, Oakland, NJ, 07436","[41.0146556, -74.2542602]"
117,Little Rockers - music & arts play center,Music Classes,http://littlerockersmusic.com/,6,"166 Patterson Ave, Shrewsbury, NJ, 07702","[40.3340040900188, -74.0713039720062]"
118,Tiny Greenhouse,Music Classes,http://tinygreenhousejc.com/,2013338450,"498 Jersey Ave, Jersey City, NJ, 07302","[40.7212653, -74.0463542]"


## Google Sports

In [9]:
url = 'https://www.google.com/search?rlz=1C1CHBF_enUS853US853&sz=0&tbm=lcl&ei=05bdXaEyoqX9BoGbkuAK&q=kids+sports+classes+new+jersey&oq=kids+sports+classes+new+jersey&gs_l=psy-ab.12...13265.16654.0.22362.7.7.0.0.0.0.83.504.7.7.0....0...1c.1.64.psy-ab..0.0.0....0.EEUl6U13Yy0#rlfi=hd:;si:;mv:[[41.1122716,-73.93472659999999],[40.3136717,-74.6914486]];tbs:lrf:!1m4!1u3!2m2!3m1!1e1!2m1!1e3!3sIAE,lf:1,lf_ui:3'
browser.visit(url)

browser.find_by_name('btnG').first.click()

html = browser.html
soup = BeautifulSoup(html, 'html.parser')
time.sleep(5)

Data = soup.find_all("div",attrs = {"class": "VkpGBb"})

print(len(Data))
top_result = []
    
for tr in Data:
    dataframe = {}
    dataframe["name"] = (tr.find("div",attrs = {"class": "dbg0pd"})).text.replace('\n', ' ')
    #dataframe["Reviews"] = (tr.find('span', {'class':'BTtC6e'}))
    top_result.append(dataframe)
print(top_result)

22
[{'name': 'Kiddie Academy Of West Caldwell'}, {'name': 'Life Time Athletic'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'The Little Gym of Montclair'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'Karate For Youth Child Care'}, {'name': 'Riverside Kids Gym'}]


In [10]:
Data = []
geolocator = Nominatim()
geolocator = Nominatim()
pageData = soup.find_all(class_='std')
#print( soup.find("div", class_="std")["aria-label"] )

print(len(pageData))
for tr in pageData:
    
    pLink = str(tr.find('a')['href'])
    print(pLink)

top_result = []
list1 = [2,4,6,8,10,12] 
for x in (list1):
    prePage = x
    #print(prePage)
    startPage = "start="+ str(prePage) +"0"
    #print(startPage)
    replacePage = "start="+ str(prePage+2) +"0"
    #print(replacePage)
    pLink = pLink.replace(startPage,replacePage)
    pageLink = "https://www.google.com" + pLink
    #print(pageLink)
    browser.visit(pageLink)

  
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    Data = soup.find_all("div",attrs = {"class": "VkpGBb"})

    print(len(Data))
    
    for tr in Data:
        
        dataframe = {}
        dataframe["name"] = (tr.find("div",attrs = {"class": "dbg0pd"})).text.replace('\n', ' ')
        #dataframe["Reviews"] = (tr.find('span', {'class':'BTtC6e'}))
        dataframe["type"] = "Sports Classes"
        try:
            dataframe["url"] = (tr.find("a",attrs = {"class": "yYlJEf L48Cpd"})['href'])
            
            print(dataframe["url"])
        except:
            print('Cant Find the url')
            
        
        PhoneData = tr.find('span').text.split(' ')
        #print(PhoneData)
        lPhoneData = len(PhoneData)
        phoneNo = PhoneData[lPhoneData-2]+ PhoneData[lPhoneData-1]
        phoneNo = re.sub("\D", "", phoneNo)
        dataframe["phoneNo"] = phoneNo
        
        dirClass = (tr.find('div', {'class':'UbRuwe'}))
        #print(dirClass)
        if dirClass != None :
        #if hasattr('a', 'cursor:pointer'):
            rawAddress = (tr.find("a",attrs = {"class": "yYlJEf VByer"})["data-url"])
            
            orgAddress = rawAddress.split('/')[4].strip()
            #print(orgAddress)
            orgAddress = orgAddress.replace('+',' ')
            orgAddress = orgAddress.split(',')
            
            #print(orgAddress)
            lenAddress = len(orgAddress)
            
            if lenAddress == 4:
                ZipCode = orgAddress[3].split(' ')
                #print(ZipCode)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[1]+', ' + orgAddress[2]+', ' + ZipCode[1]+', ' + ZipCode[2]
                #dataframe["city"] = orgAddress[2]
                #dataframe["state"] = ZipCode[1]
                #if len(ZipCode) > 2:
                    #dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
                    
            if lenAddress > 4 :
                ZipCode = orgAddress[lenAddress-1].split(' ')
                #print(ZipCode)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[lenAddress-3]+', '+orgAddress[lenAddress-2]+', '+ ZipCode[1]+', '+ZipCode[2]
                #dataframe["city"] = orgAddress[lenAddress-2]
                #dataframe["state"] = ZipCode[1]
                #if len(ZipCode) > 2:
                    #dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
            
                
            if lenAddress < 4 :
                ZipCode = orgAddress[lenAddress-1].split(' ')
                #print(orgAddress)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[lenAddress-2]+', '+orgAddress[lenAddress-1]+', '+ ZipCode[1]+', '+ZipCode[2]
                #dataframe["city"] = orgAddress[lenAddress-1]
                #dataframe["state"] = ZipCode[1]
                #if len(ZipCode) > 2:
                    #dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
            
                #print(orgAddress)
                    
            try:
                locForLatLong = dataframe["address"]# + dataframe["city"]
                #print(locForLatLong)
                location = geolocator.geocode(locForLatLong, timeout=10)
                #print(location.address)
                #print((location.latitude,location.longitude))
                dataframe["location"] = [location.latitude,location.longitude]
                
            except AttributeError:
                print('Cant Find the address')
                
        top_result.append(dataframe)

#print(top_result)
df3=pd.DataFrame(top_result)



1
/search?q=kids+sports+classes+new+jersey&rlz=1C1CHBF_enUS853US853&sz=0&tbm=lcl&ei=waTuXafWH6a4ggfu36qICA&start=20&sa=N&ved=0ahUKEwjnuufLqqnmAhUmnOAKHe6vCoEQ8tMDCIgC


C:\Users\13157\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  
C:\Users\13157\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the defaul

20
https://www.littlesport.net/
Cant Find the address
http://www.randolphgym.com/
http://www.otbathletics.com/
Cant Find the address
http://www.kidville.com/closter
http://www.i9sports.com/
http://www.tlglivingstonnj.com/
https://www.newyorksportsclubs.com/clubs/marlboro
Cant Find the address
http://www.yogaandhealingcenter.com/
http://www.edgewrestling.com/
http://www.scorpionswrestling.com/
Cant Find the address
http://www.madisonareaymca.org/
https://britishswimschool.com/region/hudson-waterfront
http://nj360fitness.com/
http://www.thelittlegym.com/seagirtnj
Cant Find the address
https://www.metroymcas.org/locations/sussex-county-ymca/sussex-county-ymca/?back=locations
Cant Find the address
http://ussportsinstitute.com/
http://www.worldclassmartialarts.net/
http://www.ffmanj.com/
http://www.centercourtacademy.com/
http://www.vizzio.com/
20
http://tumbletechsgymnastics.com/
http://www.atlantissportsclubs.com/
http://www.ajwmartialartsacademy.com/
http://www.gymnastikanj.com/
http://w

In [11]:
#df.to_excel("outputSports.xlsx")

# creating a dataframe for sports
df3


,name,type,url,phoneNo,address,location
0,LittleSport,Sports Classes,https://www.littlesport.net/,,"120 E Kings Hwy %23403, Maple Shade Township...",NaN
1,Randolph Gymnastics,Sports Classes,http://www.randolphgym.com/,7,"3 Middlebury Blvd, Randolph, NJ, 07869","[40.8627742620884, -74.6140286211776]"
2,Out of the Box Athletics | Cedar Knolls CrossFit,Sports Classes,http://www.otbathletics.com/,10,"45 Horse Hill Rd Suite 105C, Cedar Knolls, N...",NaN
3,Kidville,Sports Classes,http://www.kidville.com/closter,2019831551,"53 Vervalen St, Closter, NJ, 07624","[40.9704506177326, -73.9582628723433]"
4,i9 Sports,Sports Classes,http://www.i9sports.com/,,"143 Coe Ave, Hillside, NJ, 07205","[40.6910877663854, -74.2200071682221]"
5,The Little Gym of Livingston,Sports Classes,http://www.tlglivingstonnj.com/,9734221722,"4215 Town Center Way, Livingston, NJ, 07039","[40.7969318, -74.3139108]"
6,Boston Sports Clubs,Sports Classes,https://www.newyorksportsclubs.com/clubs/marlboro,7325364141,"34 Route 9 North, Morganville, NJ, 07751",NaN
7,Yoga & Healing Center,Sports Classes,http://www.yogaandhealingcenter.com/,7,"1744 E 2nd St, Scotch Plains, NJ, 07076","[40.6466298571429, -74.3989438571429]"
8,Edge Wrestling,Sports Classes,http://www.edgewrestling.com/,,"9 Whippany Rd, Whippany, NJ, 07981","[40.821307, -74.412318]"
9,Scorpions Wrestling School,Sports Classes,http://www.scorpionswrestling.com/,8622108046,"UT 38-39A Ste 7, Fairfield, NJ, 07004",NaN


## Merged all three dataframe into one DF

In [12]:
# merging all 3 dataframe for music, Dannce and Sports

df = df1.append(df2, ignore_index=True)
df = df.append(df3, ignore_index=True)

# Cleaning the dataframet to drop null values for location

df = df.dropna(axis=0, subset=['location','url'])
df = df[df.location != 0]
df = df[df.url != 0]
df

,name,type,url,phoneNo,address,location
0,Newark School of the Arts,Dance Classes,http://www.newarkschoolofthearts.org/,,"89 Lincoln Park, Newark, NJ, 07102","[40.7256942631579, -74.1794780526316]"
1,Dynamic Dance Academy,Dance Classes,http://dynamicdancenj.com/,,"546 Bloomfield Ave, Verona, NJ, 07044","[40.8345572, -74.2564094]"
2,Dance Starz Academy of NJ,Dance Classes,http://dancestarznj.net/,,"496 Inman Ave, Rahway, NJ, 07065","[40.6001789, -74.3113377]"
3,Monroe Street Movement Space,Dance Classes,http://www.movementspace.com/,,"720 Monroe St, Hoboken, NJ, 07030","[40.7466675918367, -74.0378767142857]"
5,Nritya Creations Academy of Dance,Dance Classes,http://www.nrityacreations.com/,,"37 Gill Ln, Iselin, NJ, 08830","[40.5689118285714, -74.3237624857143]"
...,...,...,...,...,...,...
291,Training4Life Martial Arts Academy,Sports Classes,http://www.training4lifema.com/,3,"681 NJ-23, Pompton Plains, NJ, 07444","[40.9721662413777, -74.2874075937022]"
293,Lakeland Hills Family YMCA,Sports Classes,http://www.lakelandhillsymca.com/,1030,"100 Fanny Rd, Mountain Lakes, NJ, 07046","[40.896903, -74.41785]"
294,Dover Boxing Club,Sports Classes,http://www.doverboxingclub.com/,9739451513,"33 Baldwin Rd, Parsippany, NJ, 07054","[40.86188265, -74.3975818501908]"
295,Branchburg Sports Complex,Sports Classes,http://www.branchburgsportscomplex.com/,10,"47 Readington Rd, Branchburg, NJ, 08876","[40.5977632, -74.6929397301492]"


In [13]:
# exporting to excel

df.to_excel("outputKidsActivities.xlsx") 

In [14]:
# reseting the index

df=df.reset_index()
df.head()

,index,name,type,url,phoneNo,address,location
0,0,Newark School of the Arts,Dance Classes,http://www.newarkschoolofthearts.org/,,"89 Lincoln Park, Newark, NJ, 07102","[40.7256942631579, -74.1794780526316]"
1,1,Dynamic Dance Academy,Dance Classes,http://dynamicdancenj.com/,,"546 Bloomfield Ave, Verona, NJ, 07044","[40.8345572, -74.2564094]"
2,2,Dance Starz Academy of NJ,Dance Classes,http://dancestarznj.net/,,"496 Inman Ave, Rahway, NJ, 07065","[40.6001789, -74.3113377]"
3,3,Monroe Street Movement Space,Dance Classes,http://www.movementspace.com/,,"720 Monroe St, Hoboken, NJ, 07030","[40.7466675918367, -74.0378767142857]"
4,5,Nritya Creations Academy of Dance,Dance Classes,http://www.nrityacreations.com/,,"37 Gill Ln, Iselin, NJ, 08830","[40.5689118285714, -74.3237624857143]"


In [15]:
# renaming the column to _id

df = df.rename(columns={'index':'_id'})
df = df.drop(df.index[0])
#df = df.drop(columns=['Reviews'])
#df = df.drop(columns=['_id'])
df.head()

,_id,name,type,url,phoneNo,address,location
1,1,Dynamic Dance Academy,Dance Classes,http://dynamicdancenj.com/,,"546 Bloomfield Ave, Verona, NJ, 07044","[40.8345572, -74.2564094]"
2,2,Dance Starz Academy of NJ,Dance Classes,http://dancestarznj.net/,,"496 Inman Ave, Rahway, NJ, 07065","[40.6001789, -74.3113377]"
3,3,Monroe Street Movement Space,Dance Classes,http://www.movementspace.com/,,"720 Monroe St, Hoboken, NJ, 07030","[40.7466675918367, -74.0378767142857]"
4,5,Nritya Creations Academy of Dance,Dance Classes,http://www.nrityacreations.com/,,"37 Gill Ln, Iselin, NJ, 08830","[40.5689118285714, -74.3237624857143]"
5,6,Center Stage Dance Studio,Dance Classes,http://www.centerstagedancenj.com/,,"176 NJ-17, Paramus, NJ, 07652","[40.9334675310707, -74.0705435182855]"


In [16]:
# adding to directory

split_data = df.to_dict('records')
split_data

[{'_id': 1,
  'name': 'Dynamic Dance Academy',
  'type': 'Dance Classes',
  'url': 'http://dynamicdancenj.com/',
  'phoneNo': '',
  'address': ' 546 Bloomfield Ave,  Verona, NJ, 07044',
  'location': [40.8345572, -74.2564094]},
 {'_id': 2,
  'name': 'Dance Starz Academy of NJ',
  'type': 'Dance Classes',
  'url': 'http://dancestarznj.net/',
  'phoneNo': '',
  'address': ' 496 Inman Ave,  Rahway, NJ, 07065',
  'location': [40.6001789, -74.3113377]},
 {'_id': 3,
  'name': 'Monroe Street Movement Space',
  'type': 'Dance Classes',
  'url': 'http://www.movementspace.com/',
  'phoneNo': '',
  'address': ' 720 Monroe St,  Hoboken, NJ, 07030',
  'location': [40.7466675918367, -74.0378767142857]},
 {'_id': 5,
  'name': 'Nritya Creations Academy of Dance',
  'type': 'Dance Classes',
  'url': 'http://www.nrityacreations.com/',
  'phoneNo': '',
  'address': ' 37 Gill Ln,  Iselin, NJ, 08830',
  'location': [40.5689118285714, -74.3237624857143]},
 {'_id': 6,
  'name': 'Center Stage Dance Studio',
 

In [17]:
df.dtypes

_id          int64
name        object
type        object
url         object
phoneNo     object
address     object
location    object
dtype: object

In [18]:
# Set up the MongoDB connection through pymongo

myclient = pymongo.MongoClient("mongodb://localhost:27017/")

# Create DB

db = myclient["KidsRidedb3"]

# Create collection and insert all the data into the MongoDB

mycol = db["activityData20"]
mycol.drop()
mycol.insert_many(split_data)